# Sorting out basic design issues

## Running the model for 10 bidders and 10 users

In [50]:
import numpy as np
import random
from collections import Counter

class User:
    def __init__(self):
        self.probability2 = random.uniform(0, 1)

    def show_ad(self):
        return  np.random.choice((True, False), p = [self.probability2, 1-self.probability2])

class Auction:
    
    def __init__(self, users, bidders):
        self.users = users
        self.bidders = bidders
        self.auction_history = []
        self.balances = {id(bidder): 0 for bidder in bidders} ########### NEW LINE
        print("*"*100, '\n')
        print('Beginning balances : ',self.balances, '\n','-'*100)
        
        
    def execute_round(self):
        
        ##### if the -ve balance of ALL bidders is < -1000, then break
        
        all_invalid_bidders = any(x > -1000 for x in self.balances.values())
        
        while not all_invalid_bidders:
            print("no bidders are qualifed as all balances are below -1000")
            break
        
        else:
            # print('starting balances2: ', self.balances2, '\n','-'*100)
            bids_dict = {}
            bids_list = []
            bids_list_raw = []

            # 1. SELECT USER (from the pool of users) ----------------------------------

            chosen_user = random.randint(0, len(self.users)-1)
            print('Selected User: ', id(chosen_user), '\n','-'*100) 

            # 2. COLLECT BIDS (send to all bidders) --------------------------------------------
            for bidder in self.bidders:
                #if self.balances[bidder] > -1000:
                #balances = {bidder: 0 for bidder in bidders} ########### NEW LINE
                if self.balances[id(bidder)] > -1000:
                    bids_dict[bidder] = bidder.bid(chosen_user)
                    bidding_round = bidder.bidding_round

                    self.auction_history.append((id(bids_dict[bidder]), bids_dict[bidder])) #bidder
                    highest_bid = 0
                    winning_price = 0
                else:
                    print("Bidder ", id(bidder), "balance is less than -1000. Cannot continue bid" )
     #               #raise Exception("Bidder's balance is less than -1000. Cannot continue bid")
                    continue

            print("Current bidding Round: ", bidding_round, '\n','-'*100)

            # 3. DETERMINE WINNING BID --------------------------------------------
            for bidder, bid_value in bids_dict.items():

                bids_list_raw.append((bidder, bid_value))
                ## only needed to track a human readable bidder id
                bids_list.append((id(bidder), bid_value))


    #        print('bids list', bids_list, '\n','-'*100)

            ###### SORT THE Bids List based on price ##########################################
            sorted_list = sorted(bids_list, key=lambda t:t[1])
            print('Sorted bids list', sorted_list, '\n','-'*100 )

            #------------- for analysis - creating a copy with id(bidder) --------------
            sorted_list_raw = sorted(bids_list_raw, key=lambda t:t[1])
    #        print('sorted bids list RAW', sorted_list_raw, '\n','-'*100 )


            ####### Select second highest price ###############################################
            if len(sorted_list) > 1:
                winning_price = (sorted_list_raw)[-2][1]
                winning_bidder = (sorted_list_raw)[-1][0]
            else:
                winning_price = (sorted_list_raw)[0][-1]
                winning_bidder = (sorted_list_raw)[0][0]

    #        print('Winning Price: ', winning_price, '\n','-'*100)
    #        print('winning bidder', winning_bidder, '\n','-'*100)

            ######## Determine if there are more than 1 bidders with same price ######################

            multiple_winning_bidders = [tup for tup in sorted_list_raw if tup[1] == winning_price]

            if len(multiple_winning_bidders)>1:
                print('List of biders that submitted similar bids: ' , multiple_winning_bidders, '\n','-'*100)
                randindex = random.randint(0, len(multiple_winning_bidders)-1)
                winning_bidder = multiple_winning_bidders[randindex][0]
                print('Winning bidder from list of similar priced bids: ', winning_bidder, '\n','-'*100)
            else: 
                pass
                #winning_bidder = multiple_winning_bidders[0][0]

            print('Winning Bidder (FINAL): ', id(winning_bidder), '\n','-'*100)
            print('Winning Price (FINAL): ', winning_price, '\n','-'*100)

            ############################################################################################
            # 4. Validate USER CLICK after SHOW AD  --------------------------------------------
            ad_result = self.users[chosen_user].show_ad()
            print('User Clicked on Ad?: ', ad_result,'\n','-'*100)

            # 5.NOTIFY BIDDER & UPDATE BALANCES --------------------------------------------

            print("Bidding Outcomes", '\n','-'*100)
            for bidder in self.bidders:
                if bidder == winning_bidder and ad_result == True:
                    print(id(bidder), "bidder == winning_bidder and ad_result == True:")
                    self.balances[id(bidder)] -= winning_price
                    self.balances[id(bidder)] += 1
                    # balances[bidder] -= winning_price
                    # balances[bidder] += 1                

                    bidder.notify(True, winning_price, ad_result)

                elif bidder == winning_bidder and ad_result == False:
                    print(id(bidder), "bidder == winning_bidder and ad_result == False")
                    self.balances[id(bidder)] -= winning_price
                    # balances[bidder] -= winning_price

                    bidder.notify(True, winning_price, ad_result)

                else:
                    print(id(bidder), "Did not win bid - only notify")
                    bidder.notify(False, winning_price, None)
            print('-'*100, '\n','balances at end of bidding round', self.balances,'\n','-'*100)
                # print('balances at end of bidding round', balances,'\n','-'*100)
        
class Bidder:
    
    #### maintain account balance here as well?

    # BIDDER CREATION -------------- initiated by GAME runner / Developer ------------------------
    
    def __init__(self, num_users, num_rounds):

        self.num_users = num_users
        self.num_rounds = num_rounds
        self.bidding_round = 0

        self.bid_y_n = True
        self.bid_participate = {i: 0 for i in range(num_users)} #whether bidder bids or not (USER: BID COUNT)
        self.bid_wins_no_click = {i: 0 for i in range(num_users)} # if bidder wins
        self.bid_win_user_clicks = {i: 0 for i in range(num_users)} # if user clicks
        self.bid_lost = {i: 0 for i in range(num_users)}
        
        #Transaction records
        self.bid_participate_history = [] # maintains bidderid, userid, bidding round, bid price
        self.bid_win_history = [] # won bid
        self.bid_win_user_click_history = [] # won bid and user clicked
        self.bid_lost_history = [] # lost bid, but know the user and price
        self.bid_price = 0
        
        ########### New tables
        self.bid_win_click_no = []
        self.bid_win_click_yes = []
        self.bid_lost_list = []
        self.bid_360 = []
        
        ############
        #? Winning bid price / user (because you get notified of winning bids) -- 
        ###### you know the user and winning bid (just not the click result)
        
    # SUBMIT BID  ------------ initiated by auction ------------------------------------
    
    def bid(self, user_id):
        self.bidding_round += 1
        #self.bid_participate = 0
        self.user_id = user_id
        default_bid_price = random.randint(0, 10000)/200000 #random.uniform(0, 1)
        # bid_price = self.default_bid_price + (self.bidding_round)*0.1
        #self.bid_price = default_bid_price
        self.bid_price = self.bid_strategy(user_id)
        print('bid price from STRATEGY', self.bid_price, '\n','-'*100 )
        #bid_price = 0.5
        self.bid_participate[self.user_id] = (self.bid_participate).get(self.user_id, 0)+1
        self.bid_participate_history.append([id(self), self.user_id, self.bidding_round, self.bid_price])
        return self.bid_price
    
#    def bid_strategy(self):
        
        #self.bid_360.extend(self.bid_win_history, self.bid_lost_history)
        # self.bid_360 = [item for sublst in zip(self.bid_win_history,  self.bid_lost_history) for item in sublst]

        # NOTIFIED OF BID OUTCOME ------------ initiated by auction ------------------------------------
    
    def notify(self, auction_winner, price, clicked):
        bid_y_n_won = 0
        self.auction_winner = auction_winner
        self.price = price
        self.clicked = clicked
        
        if auction_winner == True and clicked == True:
            self.bid_win_user_clicks[self.user_id] = (self.bid_win_user_clicks).get(self.user_id, 0)+1
            #self.bid_win_user_click_history.append([self.user_id, self.bidding_round])
            
            self.bid_win_click_yes.append(['BWCY', self.bidding_round, id(self), self.user_id, self.auction_winner,self.bid_price, self.price, self.clicked])
            
            self.bid_360.append(['BWCY', self.bidding_round, id(self), self.user_id, self.auction_winner,self.bid_price, self.price, self.clicked])

        elif auction_winner == True and clicked == False:
            #self.bid_participate_history.append([self.user_id, bid_y_n_won+1])
            self.bid_wins_no_click[self.user_id] = (self.bid_wins_no_click).get(self.user_id, 0)+1
            # self.bid_win_history.append([self.user_id, bid_y_n_won+1])

#            self.bid_win_history.append([self.bidding_round, self.user_id, self.auction_winner,self.price, self.clicked])
            
            self.bid_win_click_no.append(['BWCN', self.bidding_round, id(self), self.user_id, self.auction_winner,self.bid_price, self.price, self.clicked])
            
            self.bid_360.append(['BWCN', self.bidding_round, id(self), self.user_id, self.auction_winner,self.bid_price, self.price, self.clicked])
        
        else:
            self.bid_lost[self.user_id] = (self.bid_lost).get(self.user_id, 0)+1

#            self.bid_lost_history.append([self.bidding_round, self.user_id, self.auction_winner,self.price, self.clicked])

            self.bid_lost_list.append(['BLOS',self.bidding_round, id(self), self.user_id, self.auction_winner,self.bid_price, self.price, self.clicked])

            self.bid_360.append(['BLOS',self.bidding_round, id(self), self.user_id, self.auction_winner,self.bid_price, self.price, self.clicked])
    
    def bid_strategy(self, user_id):

        base_bid_price = random.randint(0, 10000)/200000 #random.uniform(0, 1)
        
        trx_master = [x for x in self.bid_360]

        for item in trx_master:
        
        ########## Observe: Start Slow for the first 10 rounds  #############
        
            if item[1] < 10:
                self.bid_price = 0.01
            
        ########## Next 40 rounds, match max price + 10%  #############
        
            
            elif item[1] > 10 and item[1] < 50:
                self.bid_price = max(self.bid_360, key=lambda x: x[6])[6]*1.10
                
            
            else:
                self.bid_price = base_bid_price
            
        ########### Review past 10 losses where loss > 10% to winning bid #########
        
        return self.bid_price
        

    
#         ########### Setting min bid above average difference of loss for prior 10 bids #######
#         ########## review past 10 bids (where you lost) where loss % was greater than 10%
#         ########### then adjust you next bid to in increase by that amount
        
#         # filter last 10 losses  
#         a = [x for x in self.bid_360[-10:] if x[0] == 'BLOS' and (x[6]-x[5]) > (.1*x[5])]
        
#         # calculated the average percentage loss (> 10%)
#         avg_loss_10percent = ((sum(i[6] for i in a) - sum(i[5] for i in a))/(max(1, len(a))))
        
#         #adjust bid price
#         self.bid_price = base_bid_price + avg_loss_10percent
    
#         ########### Focus on most successful user #############
        
#         a = [x for x in self.bid_360[-100:] if x[0] == 'BWCY']
#         #print(a)
#         b = (Counter([x[3] for x in a]))
#         #print(b)
#         # filter for success > 5 times
#         c = { k: v for k, v in b.items() if v >= 5 }
#         # get max user (most successful thus far)
#         d = max(c, key=c.get, default=0) # get user with max wins
#         self.bid_price = base_bid_price*2
        

        
        ########## Aggresive based on win-loss ratio #####
        
        
        ######### Find Max bid thus far and match is ######
        
        
        ######## Click Ratio < relative percentage of bidders, then > 20%
        
        
        ####### Strategy in case balance drops below < -500
        
        
        
        
        
        
       
        

In [51]:
#b1, b2, b3 = Bidder(1,10), Bidder(1,10), Bidder(1,10)
b1, b2, b3, b4, b5, b6, b7, b8, b9, b10 = Bidder(1,10), Bidder(1,10), Bidder(1,10), Bidder(1,10), Bidder(1,10), Bidder(1,10), Bidder(1,10), Bidder(1,10), Bidder(1,10), Bidder(1,10)
auction = Auction( [User()for i in range(10)], [b1, b2, b3, b4, b5, b6, b7, b8, b9, b10])
#auction = Auction( [User(), User()], [b1, b2, b3])

auction.execute_round()
# auction.execute_round()
# auction.execute_round()
# auction.execute_round()

**************************************************************************************************** 

Beginning balances :  {140631874982192: 0, 140631874985216: 0, 140631874984304: 0, 140631874983968: 0, 140631874985792: 0, 140631874983776: 0, 140631874985696: 0, 140631874985408: 0, 140631874982768: 0, 140631874982288: 0} 
 ----------------------------------------------------------------------------------------------------
Selected User:  140632127498800 
 ----------------------------------------------------------------------------------------------------
bid price from STRATEGY 0 
 ----------------------------------------------------------------------------------------------------
bid price from STRATEGY 0 
 ----------------------------------------------------------------------------------------------------
bid price from STRATEGY 0 
 ----------------------------------------------------------------------------------------------------
bid price from STRATEGY 0 
 ---------------------

In [47]:
[auction.execute_round() for i in range(100)]

Selected User:  140632127498576 
 ----------------------------------------------------------------------------------------------------
bid price from STRATEGY 0.01 
 ----------------------------------------------------------------------------------------------------
bid price from STRATEGY 0.01 
 ----------------------------------------------------------------------------------------------------
bid price from STRATEGY 0.01 
 ----------------------------------------------------------------------------------------------------
bid price from STRATEGY 0.01 
 ----------------------------------------------------------------------------------------------------
bid price from STRATEGY 0.01 
 ----------------------------------------------------------------------------------------------------
bid price from STRATEGY 0.01 
 ----------------------------------------------------------------------------------------------------
bid price from STRATEGY 0.01 
 ------------------------------------------

[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None]

# Consolidate Metrics: All Bidders

## Overall

In [19]:
# Summary Counts
print('Bidding Rounds', b1.bidding_round)

BWCY = (len(b1.bid_win_click_yes), len(b2.bid_win_click_yes), \
      len(b3.bid_win_click_yes), len(b4.bid_win_click_yes),  len(b5.bid_win_click_yes), len(b6.bid_win_click_yes), \
      len(b7.bid_win_click_yes), len(b8.bid_win_click_yes), len(b9.bid_win_click_yes), len(b10.bid_win_click_yes))

sum(BWCY)
print('BWCY - Bid Win - Click YES: Count', sum(BWCY))

BWCN = (len(b1.bid_win_click_no),len(b2.bid_win_click_no),len(b3.bid_win_click_no)\
     ,len(b4.bid_win_click_no),len(b5.bid_win_click_no),len(b6.bid_win_click_no),len(b7.bid_win_click_no),\
    len(b8.bid_win_click_no),len(b9.bid_win_click_no),len(b10.bid_win_click_no))

sum(BWCN)

print('BWCN - Bid Win - Click NO: Count',sum(BWCN))

# GENERAL CLICK RATIO

print('Overall Click Ratio ', sum(BWCY) / (sum(BWCY)+sum(BWCN)) )

# BLOS = NOT RELEVANT
# print('BLOS - Bid LOST: Count',len(b1.bid_lost_list))

# print('Balances', auction.balances)

Bidding Rounds 101
BWCY - Bid Win - Click YES: Count 68
BWCN - Bid Win - Click NO: Count 33
Overall Click Ratio  0.6732673267326733


# Bid Refining - Filters

## Observe first 10 rounds

In [47]:
trx_master = [x for x in self.bid_360]

for item in trx_master:

    if item[1] < 10:
        self.bid_price = 0.01
    else:
        self.bid_price = base_bid_price

# trx_master = [x for x in b1.bid_360]

# for item in trx_master:
#     if item[1] < 10:
#         print('hey')
#     else:
#         print('hey no')

hey
hey
hey
hey
hey
hey
hey
hey
hey
hey no
hey no
hey no
hey no
hey no
hey no
hey no
hey no
hey no
hey no
hey no
hey no
hey no
hey no
hey no
hey no
hey no
hey no
hey no
hey no
hey no
hey no
hey no
hey no
hey no
hey no
hey no
hey no
hey no
hey no
hey no
hey no
hey no
hey no
hey no
hey no
hey no
hey no
hey no
hey no
hey no
hey no
hey no
hey no
hey no
hey no
hey no
hey no
hey no
hey no
hey no
hey no
hey no
hey no
hey no
hey no
hey no
hey no
hey no
hey no
hey no
hey no
hey no
hey no
hey no
hey no
hey no
hey no
hey no
hey no
hey no
hey no
hey no
hey no
hey no
hey no
hey no
hey no
hey no
hey no
hey no
hey no
hey no
hey no
hey no
hey no
hey no
hey no
hey no
hey no
hey no
hey no


In [45]:
a = [x for x in b1.bid_360[-100:] if 0 < x[1] < 50]

for item in a:
    if item[1] < 10:
        print('hey')
    else:
        print('hey no')

# b = filter(lambda x: x[1] < 10, a)
# print(b)

hey
hey
hey
hey
hey
hey
hey
hey
hey no
hey no
hey no
hey no
hey no
hey no
hey no
hey no
hey no
hey no
hey no
hey no
hey no
hey no
hey no
hey no
hey no
hey no
hey no
hey no
hey no
hey no
hey no
hey no
hey no
hey no
hey no
hey no
hey no
hey no
hey no
hey no
hey no
hey no
hey no
hey no
hey no
hey no
hey no
hey no


## Loss by > 10%

In [8]:
# lost the bid by a price > 10%

# filter last 10 bids for those you have lost 
a = [x for x in b1.bid_360[-10:] if x[0] == 'BLOS' and (x[6]-x[5]) > (.1*x[5])]
#print(a)

# calculated the average percentage loss (> 10%)
b = ((sum(i[6] for i in a) - sum(i[5] for i in a))/(max(1, len(a))))

print(b)

# then set bid_price = 

# [0]Category, [1]self.bidding_round, [2]id(self), [3]self.user_id, 
# [4]self.auction_winner, [5]#self.bid_price, [6]self.winning price, [7]self.clicked

0.0256738888888889


## Most successful user

In [11]:
# lost the bid by a price > 10%
from collections import Counter

a = [x for x in b1.bid_360[-1000:] if x[0] == 'BWCY']
#print(a)
b = (Counter([x[3] for x in a]))
c = { k: v for k, v in b.items() if v >= 3 }
#print(b)
d = max(c, key=c.get, default = 0) # get user with max wins
print(d)
#print(c)

#print(max(b, key=b.get))

# then set bid_price = 

# [0]Category, [1]self.bidding_round, [2]id(self), [3]self.user_id, 
# [4]self.auction_winner, [5]#self.bid_price, [6]self.winning price, [7]self.clicked

0


## Wait certain rounds, then accelerate

In [35]:
#a = [x for x in b1.bid_360[-100:] if x[0] == 'BWCN'] # < ]
a = [x for x in b1.bid_360[-100:] if 100 < x[1] < 200] #'BWCN'] # < ]
print(a)
     # then take default bid probability

[['BLOS', 101, 140568245737168, 6, False, 0.04786, 0.08659, None]]


## Match Max and go over 10%

---

In [44]:
b1.bid_360 # Category,self.bidding_round, id(self), self.user_id, self.auction_winner,
           #self.bid_price, self.winning price, self.clicked
#[x for x in b1.bid_360]    

trx_master = [x for x in b1.bid_360]    
trx_master
    
c = max(trx_master, key=lambda x: x[6])[6]
c

0.04845

## By Bidder

### for b1

In [142]:
# Summary Counts
print('Bidding Rounds', b1.bidding_round)
print('BWCY - Bid Win - Click YES: Count', len(b1.bid_win_click_yes))
print('BWCN - Bid Win - Click NO: Count',len(b1.bid_win_click_no))
print('BLOS - Bid LOST: Count',len(b1.bid_lost_list))
print('Balances', auction.balances)

Bidding Rounds 101
BWCY - Bid Win - Click YES: Count 6
BWCN - Bid Win - Click NO: Count 5
BLOS - Bid LOST: Count 90
Balances {140186674701072: -39.189, 140186674700832: -34.350500000000004, 140186674702272: -29.9785, 140186674702032: -50.329, 140186674701888: -46.2295, 140186674701456: -46.038500000000006, 140186674702896: -22.3095, 140186674701408: -32.791, 140186947654512: -36.7655, 140186947654224: -35.323499999999996}


## for b2

In [26]:
# Summary Counts
print('Bidding Rounds', b2.bidding_round)
print('BWCY - Bid Win - Click YES: Count', len(b2.bid_win_click_yes))
print('BWCN - Bid Win - Click NO: Count',len(b2.bid_win_click_no))
print('BLOS - Bid LOST: Count',len(b2.bid_lost_list))
print('Balances', auction.balances)

Bidding Rounds 101
BWCY - Bid Win - Click YES: Count 6
BWCN - Bid Win - Click NO: Count 6
BLOS - Bid LOST: Count 89
Balances {140186667607184: 7.015412441277343, 140186667609344: 5.250825039664694, 140186667608768: 1.3441374456608397, 140186667609152: 0.5945127572589078, 140186667609536: 4.312658472669412, 140186667608480: 0.5513905656178402, 140188025144080: 5.863932297976039, 140188025146480: 3.4075567665145567, 140188025146288: 1.5182106453229096, 140188025146048: 2.6176709594806593}


## for b3

In [20]:
# Summary Counts
print('Bidding Rounds', b3.bidding_round)
print('BWCY - Bid Win - Click YES: Count', len(b3.bid_win_click_yes))
print('BWCN - Bid Win - Click NO: Count',len(b3.bid_win_click_no))
print('BLOS - Bid LOST: Count',len(b3.bid_lost_list))
print('Balances', auction.balances)

Bidding Rounds 201
BWCY - Bid Win - Click YES: Count 6
BWCN - Bid Win - Click NO: Count 15
BLOS - Bid LOST: Count 180
Balances {140208958109920: 5.306381296647432, 140208956410224: 9.097363318097536, 140208954557104: 4.6733569133451445, 140208954558256: 9.623191271869645, 140208954557296: 10.74566377202887, 140208954559600: 10.44058025297851, 140208954556768: 10.274356090469286, 140208954556912: 5.74534371340308, 140208954557680: 11.77527405632269, 140208954559408: 6.68760711988454}


---

# User Info

In [148]:
print('Bidding Rounds', b1.bidding_round)
print('BWCY - Bid Wins User Clicked', {k: v for k, v in sorted(b1.bid_win_user_clicks.items(), key=lambda item: item[0])})
print('BWCN - Bid Wins User NO Clicked', {k: v for k, v in sorted(b1.bid_wins_no_click.items(), key=lambda item: item[0])})
print('BLOS - Bids Lost', {k: v for k, v in sorted(b1.bid_lost.items(), key=lambda item: item[0])})


Bidding Rounds 201
BWCY - Bid Wins User Clicked {0: 1, 1: 1, 2: 1, 4: 1, 6: 2, 7: 3, 8: 1, 9: 3}
BWCN - Bid Wins User NO Clicked {0: 0, 1: 2, 4: 4, 7: 1, 8: 2}
BLOS - Bids Lost {0: 15, 1: 15, 2: 21, 3: 14, 4: 25, 5: 15, 6: 17, 7: 20, 8: 23, 9: 14}


In [24]:
b1.bid_360

[['BLOS', 1, 140631870820752, 0, False, 0, 0, None],
 ['BLOS', 2, 140631870820752, 9, False, 0.01, 0.01, None],
 ['BLOS', 3, 140631870820752, 8, False, 0.01, 0.01, None],
 ['BLOS', 4, 140631870820752, 5, False, 0.01, 0.01, None],
 ['BLOS', 5, 140631870820752, 2, False, 0.01, 0.01, None],
 ['BLOS', 6, 140631870820752, 9, False, 0.01, 0.01, None],
 ['BLOS', 7, 140631870820752, 4, False, 0.01, 0.01, None],
 ['BLOS', 8, 140631870820752, 8, False, 0.01, 0.01, None],
 ['BWCY', 9, 140631870820752, 8, True, 0.01, 0.01, True],
 ['BLOS', 10, 140631870820752, 4, False, 0.01, 0.01, None],
 ['BLOS', 11, 140631870820752, 5, False, 0.01896, 0.03877, None],
 ['BLOS', 12, 140631870820752, 3, False, 0.041105, 0.042475, None],
 ['BLOS', 13, 140631870820752, 2, False, 0.0097, 0.040555, None],
 ['BLOS', 14, 140631870820752, 2, False, 0.046645, 0.046645, None],
 ['BLOS', 15, 140631870820752, 0, False, 0.01983, 0.03821, None],
 ['BLOS', 16, 140631870820752, 1, False, 0.03321, 0.036455, None],
 ['BLOS', 17, 1